# Experimenting w tkinter

## Summary

It's possible to create fast, lightweight canvases using Tkinter. By construction, canvases open in a separate window. In this folder we experiment with embedding this canvas in a cell, the way that iPyCanvas does.

Tkinter canvases support styling options for drawing such as line color , marker width, and canvas background color. I suggest locking these options so the user writes with a relatively narrow (width<5) black marker on a white canvas to best suite the 

## Simple canvas

The code below outputs a canvas in a separate window.


In [ ]:
# from tkinter import * #originally wildcard import
from tkinter import Tk
from tkinter import Canvas
from tkinter import ttk
from tkinter import constants as con

def savePosn(event):
    global lastx, lasty
    lastx, lasty = event.x, event.y

def addLine(event):
    canvas.create_line((lastx, lasty, event.x, event.y))
    savePosn(event)

root = Tk()

root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)

canvas = Canvas(root)
canvas.grid(column=0, row=0, sticky=(con.N, con.W, con.E ,con.S ))
canvas.bind("<Button-1>", savePosn)
canvas.bind("<B1-Motion>", addLine)

root.mainloop()

## Output 1:

The root object with the mainloop() method will open a window where drawing takes place.

<img src="./figures/tkinter_canvas_1.png" alt="Kitten" title="A cute kitten" width="350" height="100" /> 


## Another canvas

This canvas encapsulates some of the code above so global variables are not used.

In [ ]:
# from tkinter import *
from tkinter import Tk
from tkinter import Canvas
from tkinter import ttk
from tkinter import constants as con

class Sketchpad(Canvas):
    def __init__(self, parent, **kwargs):
        super().__init__(parent, **kwargs)
        self.bind("<Button-1>", self.save_posn)
        self.bind("<B1-Motion>", self.add_line)
        
    def save_posn(self, event):
        self.lastx, self.lasty = event.x, event.y

    def add_line(self, event):
        # default option
        #self.create_line((self.lastx, self.lasty, event.x, event.y))
        
        # some other ways to instantiate this line
        self.create_line((self.lastx, self.lasty, event.x, event.y),
                         fill="black", width=1)
        self.save_posn(event)

root = Tk()
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)

# default
# sketch = Sketchpad(root)

# white background
sketch = Sketchpad(root,background="white")
sketch.grid(column=0, row=0, sticky=(con.N, con.W, con.E, con.S))

root.mainloop()

## Output 2:

With the default options.

<img src="./figures/tkinter_canvas_2.png" alt="" title="" width="350" height="100" /> 

## Output 3:

Playing with multiple options can be done like this:  

`self.create_line((self.lastx, self.lasty, event.x, event.y), fill="red", width=5)`

There are issues with wider paint brushes in that they tend to segment.  I think the color black with a width <5 is best so the OCR receives quality data.  Background color can alos play a role. Best set this to white so there is a higher contrast between writing (black) and canvas (white).

<img src="./figures/tkinter_canvas_3.png" alt="" title="" width="350" height="100" /> 

<img src="./figures/tkinter_canvas_4.png" alt="" title="" width="350" height="100" /> 

<img src="./figures/tkinter_canvas_5.png" alt="" title="" width="350" height="100" /> 

## Output 4:

Tkinter canvas with a white background.

`sketch = Sketchpad(root,background="white")`

<img src="./figures/tkinter_canvas_6.png" alt="" title="" width="350" height="100" /> 